In [1]:
#Marsha Gomez Gomez
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install whoosh

     |████████████████████████████████| 471kB 5.3MB/s 


In [3]:
# %matplotlib inline

from whoosh import index
from whoosh.fields import *
from whoosh.analysis import *
from whoosh import qparser
import os, os.path

from matplotlib import pyplot as plt
from IPython.display import display, HTML

from base64 import b64encode
from urllib.parse import quote
from io import BytesIO

BASE_DIR = '/content/gdrive/My Drive/mircv2021'

COCO_CAPTIONS_DATASET = BASE_DIR+'/data/txt/captions.properties'
COCO_ANNOTATIONS_DATASET = BASE_DIR+'/data/txt/annotations.properties'
INDEX_NAME = 'coco'
IMG_DIR = BASE_DIR+'/data/coco_img'

print('Setting Environment Variables. Done')

Setting Environment Variables. Done


In [4]:
#TODO Indexing

if not os.path.exists(BASE_DIR + "/out"):
    os.mkdir(BASE_DIR + "/out")
if not os.path.exists(BASE_DIR + "/out/whoosh"):
    os.mkdir(BASE_DIR + "/out/whoosh")

print('creating ' + INDEX_NAME + ' index...')

schema = Schema(id=ID(stored=True),
                text=TEXT(analyzer=KeywordAnalyzer(),stored=True),
                annotation=TEXT(analyzer=KeywordAnalyzer(),stored=True))
#create whoosh index
ix = index.create_in(BASE_DIR + "/out/whoosh", schema)

print('indexing done ')


creating coco index...
indexing done 


In [5]:
try:
  #initialize whoosh writer
  writer = ix.writer()

  with open(COCO_CAPTIONS_DATASET) as captions, open(COCO_ANNOTATIONS_DATASET) as annotations:
    for caption_line, annotation_line in zip(captions, annotations):
      CAPTION_ID, CAPTION_TEXT = caption_line.split("=")
      ANNOTATION_ID, ANNOTATION_TEXT = annotation_line.split("=")
      compare = CAPTION_ID.zfill(12)+".jpg"
      writer.add_document(id=CAPTION_ID.zfill(12), text=CAPTION_TEXT, annotation=ANNOTATION_TEXT)
      print(CAPTION_ID+' indexed')

  #commit writer
  writer.commit()

except:
  writer.cancel()

print('indexing done')

289343 indexed
61471 indexed
472375 indexed
520301 indexed
579321 indexed
494869 indexed
554002 indexed
78823 indexed
419974 indexed
404484 indexed
329219 indexed
68078 indexed
170893 indexed
65485 indexed
498286 indexed
424162 indexed
61108 indexed
67213 indexed
365207 indexed
131273 indexed
279278 indexed
482100 indexed
540502 indexed
127182 indexed
565776 indexed
575970 indexed
462614 indexed
407614 indexed
522007 indexed
226984 indexed
37777 indexed
491216 indexed
136355 indexed
529568 indexed
306733 indexed
42276 indexed
370818 indexed
464476 indexed
68833 indexed
433915 indexed
149222 indexed
17379 indexed
361586 indexed
127987 indexed
186632 indexed
440475 indexed
368900 indexed
9772 indexed
525083 indexed
383443 indexed
240940 indexed
435003 indexed
17899 indexed
70774 indexed
508602 indexed
123585 indexed
157807 indexed
109055 indexed
23272 indexed
134096 indexed
501523 indexed
125850 indexed
284623 indexed
245764 indexed
466156 indexed
402473 indexed
10363 indexed
172330 inde

In [6]:
def get_table(img, title, txt):
    table ='<table><td><img style="width:320px" src="' + img + '"></td>'
    table +='<td style="text-align:left"><b>' + title + '</b></p>' + txt + '</td></table>'
    return table


def display_result(img_path, title, txt):
    with open(img_path, 'rb') as f:
        dataEncoded = b64encode(BytesIO(f.read()).getvalue())
        data = 'data:image/jpg;base64,{}'.format(quote(dataEncoded))
        display(HTML(get_table(data, title, txt)))

In [18]:
#TODO Searching
#initialize whoosh searcher
searcher = ix.searcher()

#txt = "annotation:person text:people text:bike"
#txt = "annotation:person text:people bike"

#txt = "annotation:person text:people NOT text:bike"
#txt = "annotation:person text:beach OR text:bird"
txt = "annotation:giraffe text:giraffe text:giraffe"


print("query: " + txt)


#perform txt query
parser = qparser.QueryParser("text", ix.schema, group=qparser.AndGroup)
txt_query = parser.parse(txt)
res = searcher.search(txt_query) 

print(res[0])
print(res[0].score)



query: annotation:giraffe text:giraffe text:giraffe
<Hit {'annotation': 'animal giraffe animal giraffe\n', 'id': '000000152870', 'text': 'A giraffe standing on a grass covered field. A giraffe stands in an area of dead shrubbery. a single giraffe standing in tall brown grass A giraffe is standing alone in a dead patch of foliage. A single giraffe is standing in some tall brush looking all around.\n'}>
14.45241211524284


In [20]:
for hit in res:
  score = hit.score 
  id = hit["id"]
  text = hit["text"]
  filename = id+".jpg"
  filepath = IMG_DIR+"/"+filename
  title = "ID: "+ id + " SCORE: "+ str(score)
  #print(id,":",text,'score: ', score)

#display the search results
display_result(filepath,title,text)
print("filename"+filepath)


filename/content/gdrive/My Drive/mircv2021/data/coco_img/000000235057.jpg
